In [6]:
# step 0: spatially join business data to MAZs using "contains" so businesses on MAZ boundaries don't get dropped

In [7]:
import numpy as np
import pandas as pd

In [8]:
maz_bus = pd.read_csv('./mazs_TM2_v2_2_BA_contains.csv')
naics = pd.read_csv('./naics_recode.csv')

In [9]:
# drop one copy of businesses that were joined to two MAZs because they are on the boundary
print(len(maz_bus))
maz_bus_null = maz_bus[maz_bus['JOIN_FID'] == -1]
maz_bus_notnull = maz_bus[maz_bus['JOIN_FID'] != -1]
maz_bus = maz_bus_notnull.drop_duplicates('JOIN_FID', keep='last')

332461


In [10]:
# join businesses and steelhead categories
maz_bus['naicssix'] = maz_bus['NAICS'].astype(str).str[:6].astype(int)
maz_bus_null['naicssix'] = 0
maz_bus = maz_bus.append(maz_bus_null)
maz_bus_naics = pd.merge(maz_bus, naics, on='naicssix', how='left')
print(len(maz_bus_naics))

C:\Users\etheocharides\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


332386


In [160]:
# number of mazs 
len(maz_bus_naics['maz'].unique())

39726

In [161]:
# create steelhead columns and populate them with no. of employees
for i in maz_bus_naics['steelhead'].unique():
    maz_bus_naics[i] = 0
    for j in np.arange(len(maz_bus_naics)):
        if maz_bus_naics['steelhead'][j] == i:
            maz_bus_naics.loc[j,i] = maz_bus_naics['EMPNUM'][j]
maz_bus_naics.describe()

,OBJECTID_x,Join_Count,TARGET_FID,JOIN_FID,maz,ALAND10,AWATER10,blockcount,taz,partcount,...,ed_other,util,hotel,ed_high,man_bio,ag,lease,man_tech,natres,nan
count,332386.0,332386.000000,332386.000000,332386.000000,332386.000000,3.323860e+05,3.323860e+05,332386.000000,332386.000000,332386.000000,...,332386.000000,332386.000000,332386.000000,332386.000000,332386.000000,332386.000000,332386.000000,332386.000000,332386.000000,332386.0
mean,-1.0,0.974078,18958.362142,157742.327126,317667.427392,4.286485e+05,9.332319e+02,3.644537,301198.745720,1.014276,...,0.066516,0.113299,0.181166,0.255733,0.024478,0.034237,0.016385,0.404424,0.007224,0.0
std,0.0,0.158902,12089.274822,95804.235033,225270.494435,2.529834e+06,7.721203e+04,4.600316,225517.692332,0.155133,...,1.407943,12.501053,8.157885,42.299111,5.162787,2.388664,0.982098,36.237770,1.546999,0.0
min,-1.0,0.000000,0.000000,-1.000000,10001.000000,2.305000e+03,0.000000e+00,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,-1.0,1.000000,8664.000000,74512.250000,210103.000000,2.473700e+04,0.000000e+00,1.000000,200007.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,-1.0,1.000000,18121.000000,157616.500000,311860.000000,7.052100e+04,0.000000e+00,2.000000,300172.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,-1.0,1.000000,29564.000000,240754.750000,420764.000000,2.115710e+05,0.000000e+00,4.000000,400385.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,-1.0,1.000000,39725.000000,323903.000000,814506.000000,2.970693e+08,2.923688e+07,89.000000,800210.000000,14.000000,...,300.000000,5000.000000,3000.000000,17383.000000,2001.000000,900.000000,300.000000,11269.000000,700.000000,0.0


In [162]:
# clean up columns and summarize employment by MAZ
# this includes a "missing" category, needs to be addressed when revisited (for sixcat it was folded into othempn)
maz_bus_naics_col = maz_bus_naics[['maz', 'EMPNUM', 'ed_k12', 'logis', 'prof', 'serv_pers', 'ret_loc', 'health', 'ed_other', 
                               'man_lgt', 'serv_bus', 'constr', 'fire', 'eat', 'ag', 'transp', 'serv_soc', 'art_rec', 
                               'man_tech', 'info', 'ret_reg', 'gov', 'mis', 'hotel', 'natres', 'util', 'man_hvy', 'ed_high', 
                               'man_bio', 'lease']]
maz_bus_naics_sum = maz_bus_naics_col.groupby(['maz']).sum()
maz_bus_naics_sum.head()

,EMPNUM,ed_k12,logis,prof,serv_pers,ret_loc,health,ed_other,man_lgt,serv_bus,...,ret_reg,gov,mis,hotel,natres,util,man_hvy,ed_high,man_bio,lease
maz,,,,,,,,,,,,,,,,,,,,,
10001,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,12.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004,25.0,10.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10005,67.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
# check that employment totals are the same at the beginning and end
print(maz_bus['EMPNUM'].sum())
print((maz_bus_naics_sum['ed_k12'].sum()+maz_bus_naics_sum['logis'].sum()+maz_bus_naics_sum['prof'].sum()
+maz_bus_naics_sum['serv_pers'].sum()+maz_bus_naics_sum['ret_loc'].sum()+maz_bus_naics_sum['health'].sum()
+maz_bus_naics_sum['ed_other'].sum()+maz_bus_naics_sum['man_lgt'].sum()+maz_bus_naics_sum['serv_bus'].sum()
+maz_bus_naics_sum['constr'].sum()+maz_bus_naics_sum['fire'].sum()+maz_bus_naics_sum['eat'].sum()
+maz_bus_naics_sum['ag'].sum()+maz_bus_naics_sum['transp'].sum()+maz_bus_naics_sum['serv_soc'].sum()
+maz_bus_naics_sum['art_rec'].sum()+maz_bus_naics_sum['man_tech'].sum()+maz_bus_naics_sum['info'].sum()
+maz_bus_naics_sum['ret_reg'].sum()+maz_bus_naics_sum['gov'].sum()+maz_bus_naics_sum['mis'].sum()
+maz_bus_naics_sum['hotel'].sum()+maz_bus_naics_sum['natres'].sum()+maz_bus_naics_sum['util'].sum()
+maz_bus_naics_sum['man_hvy'].sum()+maz_bus_naics_sum['ed_high'].sum()+maz_bus_naics_sum['man_bio'].sum()
+maz_bus_naics_sum['lease'].sum()))

3685114.0
3685114.0


In [165]:
# check that the correct no. of MAZs come out
len(maz_bus_naics_sum)

39726

In [164]:
maz_bus_naics_sum.to_csv('./2015_employment_MAZv2.2.csv')